In [1]:
import pandas as pd
import decoupler as dc
import omnipath as op
from pypath.utils import mapping
import pypath.utils as pyp

ModuleNotFoundError: No module named 'decoupler'

In [7]:
pyp.mapping


<module 'pypath.utils.mapping' from '/home/jan/programs/mambaforge/envs/scpy/lib/python3.11/site-packages/pypath/utils/mapping.py'>

In [2]:
help(dc.translate_net)

Help on function translate_net in module decoupler.omnip:

translate_net(net: 'pd.DataFrame', columns: 'str | Iterable[str]' = ('source', 'target', 'genesymbol'), source_organism: 'str | int' = 'human', target_organism: 'str | int' = 'mouse', id_type: 'str | tuple[str, str]' = 'genesymbol', unique_by: 'Iterable[str] | None' = ('source', 'target'), **kwargs: 'dict[str, str]') -> 'pd.DataFrame'
    Translate networks between species by orthology.
    
    This function downloads orthology databases from omnipath and converts
    genes between species. The first time you run this function will take a
    while (~15 minutes) but then it stores all the information in cache for
    quick reusability.
    
    In case you need to reset the cache, you can do it by doing:
    ``rm -r ~/.pypath/cache/``.
    
    With its default parameters, this function translates almost any network
    or annotation data frame acquired by the functions in this module from
    human to mouse. For the PROGENy r

In [10]:
df = pd.read_csv('gene_names.csv')
#df.info()
hs = df[df["spec"] == "Hs"]
mm = df[df["spec"] == "Mm"]
rn = df[df["spec"] == "Rn"]


In [11]:

df = pd.read_csv('gene_names.csv')
hs = df[df['spec'] == 'Hs']
# we have >30k human gene symbols here, that's way more than
# all human primary gene symbols
hs.gene.unique().shape

human_gs = set(hs.gene.unique())
up_gs_human = mapping.translation_df('uniprot', 'genesymbol')
# here we see 20k, these are the primary human gene symbols
# of protein coding genes
up_gs_human.genesymbol.unique().shape

non_primary_human_gs = human_gs - set(up_gs_human.genesymbol.unique())
# these are the ones we don't see in UniProt
len(non_primary_human_gs)

list(non_primary_human_gs)[:100]
# few can be translated to UniProt because it is a secondary
# gene symbol:
mapping.map_name('TWISTNB', 'genesymbol', 'uniprot')
{'Q3B726'}
# but most of them are not recognized at all
mapping.map_name('NRIR', 'genesymbol', 'uniprot')
mapping.map_name('MTND4P12', 'genesymbol', 'uniprot')
primary_human_gs = human_gs & set(up_gs_human.genesymbol.unique())
len(primary_human_gs)
19137
df_h = df[df.gene.isin(primary_human_gs)]
df_h = df_h[df_h.spec == 'Hs']
df_h.shape
(19137, 2)
df_h['record'] = list(range(0, df_h.shape[0]))
# translate the human gene symbols to UniProt
df_h = df_h.merge(up_gs_human, left_on = 'gene', right_on = 'genesymbol', how = 'left')
# translate the human UniProt to mouse UniProt
human_mouse = homology.get_df(10090)
df_h_m = df_h.merge(human_mouse, left_on = 'uniprot', right_on = 'source', how = 'left')
# translate the mouse UniProt to gene symbol
up_gs_mouse = mapping.translation_df('uniprot', 'genesymbol', ncbi_tax_id = 10090)
df_h_m_gs = df_h_m.merge(up_gs_mouse, left_on = 'target', right_on = 'uniprot', how = 'left')
df_h_m_pp = df_h_m_gs[~df_h_m_gs.genesymbol_y.isna()]
# and finally, do the same with the decoupler function
df_h_m_dc = dc.translate_net(df_h, columns = 'gene', source_organism = 'human', target_organism = 'mouse')

# the translation success is 80% vs 94% in these two cases, likely because I used a rather outdated
# table from my local cache, while the decoupler solution uses Daniel's implementation with HomoloGene
# which is also outdated, we're working on a proper solution
df_h_m_pp.gene.unique().shape
(15433,)
df_h_m_pp.gene.unique().shape[0] / df_h.gene.shape[0]
0.8046
df_h_m_dc.gene.unique().shape
(18031,)
df_h_m_dc.gene.unique().shape[0] / df_h.gene.shape[0]
0.9401

NameError: name 'homology' is not defined

In [12]:
human_gs = set(hs.gene.unique())
up_gs_human = mapping.translation_df('uniprot', 'genesymbol')

non_primary_human_gs = human_gs - set(up_gs_human.genesymbol.unique())
# these are the ones we don't see in UniProt
len(non_primary_human_gs)

list(non_primary_human_gs)[:100]

mapping.map_name('TWISTNB', 'genesymbol', 'uniprot')

df_h = df[df.gene.isin(primary_human_gs)]
df_h = df_h[df_h.spec == 'Hs']
df_h.shape
(19137, 2)
df_h['record'] = list(range(0, df_h.shape[0]))

# translate the human gene symbols to UniProt
df_h = df_h.merge(up_gs_human, left_on = 'gene', right_on = 'genesymbol', how = 'left')
# translate the human UniProt to mouse UniProt
human_mouse = homology.get_df(10090)
df_h_m = df_h.merge(human_mouse, left_on = 'uniprot', right_on = 'source', how = 'left')

NameError: name 'homology' is not defined

In [13]:
df_h

,gene,spec,record,uniprot,genesymbol
0,HBG2,Hs,0,P69892,HBG2
1,HBG1,Hs,1,P69891,HBG1
2,TMSB15A,Hs,2,P0CG34,TMSB15A
3,ANLN,Hs,3,Q9NQW6,ANLN
4,TCAP,Hs,4,O15273,TCAP
...,...,...,...,...,...
19139,IRAG1,Hs,19098,Q9Y6F6,IRAG1
19140,SMIM11,Hs,19099,P58511,SMIM11
19141,RIMOC1,Hs,19100,A6NDU8,RIMOC1
19142,UPK3BL2,Hs,19101,E5RIL1,UPK3BL2


## Translate from human to Mouse

In [14]:

hs_to_mm = dc.translate_net(hs, columns="gene", source_organism="human", target_organism="mouse")

print(f"Number of genes pre translation: {hs.shape[0]}")
print(f"Number of genes post translation: {hs_to_mm.shape[0]}")

ImportError: pypath-omnipath is not installed. Please install it with: pip install git+https://github.com/saezlab/pypath.git

## Translate from rat to mouse

In [26]:
rn_to_mm = dc.translate_net(rn, columns="gene", source_organism="rat", target_organism="mouse")

print(f"Number of genes pre translation: {rn.shape[0]}")
print(f"Number of genes post translation: {rn_to_mm.shape[0]}")

Number of genes pre translation: 8372
Number of genes post translation: 3


In [27]:
# Create a list of data frames
data_frames = [hs_to_mm, hs, rn_to_mm, rn]

# Define file names for CSV files
file_names = ['hs_to_mm.csv', "hs.csv", 'rn_to_mm.csv', "rn.csv"]

# Loop through the list and save each data frame as a CSV file
for i, df in enumerate(data_frames):
    df.to_csv(file_names[i], index=False)
